In [39]:
import numpy as np
from IPython.display import display, HTML


In [40]:
import numpy as np

from scipy import sparse


def diagonal_dominant(a, b):
    matrix = np.column_stack([a, b])
    
    matrix[0], matrix[2] = matrix[2].copy(), matrix[0].copy()
    
    matrix /= matrix[:, 0].reshape((-1, 1))
    for i in range(1, 4):
        matrix[i] -= matrix[i][0] * matrix[0]

    matrix[2] += 8 * matrix[1]
    matrix[0] = matrix[0] - 11 * matrix[1] + matrix[2]
    return matrix[:, :-1], matrix[:, -1:]


def gaussian_elimination_one_step(A, b, column):
    A_copy = np.copy(A)
    b_copy = np.copy(b)
    n = len(A_copy)

    max_row = column
    for i in range(column + 1, n):
        if abs(A_copy[i][column]) > abs(A_copy[max_row][column]):
            max_row = i

    A_copy[[column, max_row]] = A_copy[[max_row, column]]
    b_copy[[column, max_row]] = b_copy[[max_row, column]]


    for i in range(column + 1, n):
        factor = A_copy[i][column] / A_copy[column][column]
        A_copy[i] -= factor * A_copy[column]
        b_copy[i] -= factor * b_copy[column]
    
    
    return A_copy, b_copy

A = np.array([
    [2.12, 0.42, 1.34, 0.88],
    [0.42, 3.95, 1.87, 0.43],
    [1.34, 1.87, 2.98, 0.46],
    [0.88, 0.43, 0.46, 4.44]
])


b = np.array([11.172, 0.115, 0.009, 9.349])




column = 0 
A_step, b_step = gaussian_elimination_one_step(A, b, 0)


print("Original Matrix:")
print(A)
print("\nConstant vector:")
print(b)
print("\nMatrix after one step of Gaussian elimination:")
print(A_step)
print("\nConstant vector after one step of Gaussian elimination:")
print(b_step)

def make_diagonally_dominant(matrix):
    A = sparse.csr_matrix(matrix)
    n = A.shape[0]
    row_sum = np.abs(A).sum(axis=1).A1 - np.abs(A.diagonal())
    diag_increment = np.maximum(0, row_sum - np.abs(A.diagonal()) + 1)
    D = sparse.diags(diag_increment)
    A_dominant = A + D
    
    return A_dominant.toarray()

def has_diagonal_dominance_scipy(matrix):
    A = sparse.csr_matrix(matrix)
    diag = A.diagonal()
    row_sum = np.abs(A).sum(axis=1).A1 - np.abs(diag)    
    return np.all(np.abs(diag) > row_sum)

A_step = make_diagonally_dominant(A)
has_diagonal_dominance_scipy(A_step)

Original Matrix:
[[2.12 0.42 1.34 0.88]
 [0.42 3.95 1.87 0.43]
 [1.34 1.87 2.98 0.46]
 [0.88 0.43 0.46 4.44]]

Constant vector:
[11.172  0.115  0.009  9.349]

Matrix after one step of Gaussian elimination:
[[ 2.12     0.42     1.34     0.88   ]
 [ 0.       3.86679  1.60453  0.25566]
 [ 0.       1.60453  2.13302 -0.09623]
 [ 0.       0.25566 -0.09623  4.07472]]

Constant vector after one step of Gaussian elimination:
[11.172   -2.09832 -7.05255  4.71157]


True

In [41]:
import numpy as np
from numpy import ndarray
np.set_printoptions(precision=5, suppress=True)




def jacobi(A, b, tolerance=1e-7, max_iterations=10000):
    global jacobi_b
    x = np.zeros_like(b, dtype=np.double)
    T = A - np.diag(np.diagonal(A))
    
    for k in range(max_iterations):
        print(f"Ітерація №{k+1}")
        x_old = x

        x = (b - np.dot(T, x)) / np.diagonal(A).reshape((-1, 1))
        
        print("Наближення")
        print(x.reshape((-1, 1)))
        
        print("Вектор нев'язки")
        vector = b - np.dot(A, x.reshape((-1, 1)))
        print(vector)
        
        if np.allclose(x_old, x, atol=tolerance, rtol=0.):
            break
        
        jacobi_b = vector
    
    return x

def seidel(A, b, tolerance=1e-6, max_iterations=10000):
    global seidel_b
    n = len(A)
    x = np.zeros_like(b, dtype=np.double)
    converge = False
    
    for k in range(max_iterations):
        if converge:
            break
        print(f"Ітерація №{k+1}")
        x_new = np.copy(x)
        
        for i in range(n):
            s1 = sum(A[i][j] * x_new[j] for j in range(i))
            s2 = sum(A[i][j] * x[j] for j in range(i + 1, n))
            x_new[i] = (b[i] - s1 - s2) / A[i][i]
        
        print("Наближення")
        print(x_new.reshape((-1, 1)))
        
        print("Вектор нев'язки")
        vector = b - np.dot(A, x_new.reshape((-1, 1)))
        converge = np.allclose(x, x_new, atol=tolerance, rtol=0.)
        print(vector)
        
        seidel_b = vector
        x = x_new
    
    return x.reshape((-1, 1))


A_origin = np.array([
    [2.12, 0.42, 1.34, 0.88],
    [0.42, 3.95, 1.87, 0.43],
    [1.34, 1.87, 2.98, 0.46],
    [0.88, 0.43, 0.46, 4.44]
])

A = np.array([[2.12, 0.42, 1.34, 0.88], [0.0, 3.8667924528301887, 1.6045283018867926, 0.25566037735849056], [0.0, 1.6045283018867926, 2.1330188679245285, -0.09622641509433955], [0.0, 0.25566037735849056, -0.09622641509433955, 4.074716981132076]])
b = np.array([[11.172], [-2.0983207547169807], [-7.05254716981132], [4.7115660377358495]])

tolerance = 1e-6
Ab = np.column_stack([A, b])


print("="*50)
print("Матриця за варіантом завдання")
print("----------------------------------------------------------------------------------------")
print(A_origin)
print("="*50)
print("Вхідна матриця (З діагональною перевагою після ітерації методу гауса)")
print("----------------------------------------------------------------------------------------")
print(A)
print("="*50)
print("Зведена матриця")
print("----------------------------------------------------------------------------------------")
print(np.column_stack([A, b]))  
print("="*50)
print("Метод Зейделя")
print("----------------------------------------------------------------------------------------")
print("Відповідь:", sei := seidel(A.copy(), b.copy(), tolerance=tolerance), sep="\n")
print("="*50)
print("Метод простих ітерацій")
print("----------------------------------------------------------------------------------------")
print("Відповідь:", jac := jacobi(A.copy(), b.copy(), tolerance=0.1), sep="\n")
print("="*50)

print(f"MSE SEIDEL: {np.mean((seidel_b) ):.5f}")
print(f"MSE Jacoby: {np.mean((jacobi_b) ):.5f}")
print("Методом зейделя:", sei, sep="\n")
print("Прості ітерації:", jac, sep="\n")



Матриця за варіантом завдання
----------------------------------------------------------------------------------------
[[2.12 0.42 1.34 0.88]
 [0.42 3.95 1.87 0.43]
 [1.34 1.87 2.98 0.46]
 [0.88 0.43 0.46 4.44]]
Вхідна матриця (З діагональною перевагою після ітерації методу гауса)
----------------------------------------------------------------------------------------
[[ 2.12     0.42     1.34     0.88   ]
 [ 0.       3.86679  1.60453  0.25566]
 [ 0.       1.60453  2.13302 -0.09623]
 [ 0.       0.25566 -0.09623  4.07472]]
Зведена матриця
----------------------------------------------------------------------------------------
[[ 2.12     0.42     1.34     0.88    11.172  ]
 [ 0.       3.86679  1.60453  0.25566 -2.09832]
 [ 0.       1.60453  2.13302 -0.09623 -7.05255]
 [ 0.       0.25566 -0.09623  4.07472  4.71157]]
Метод Зейделя
----------------------------------------------------------------------------------------
Ітерація №1
Наближення
[[ 5.26981]
 [-0.54265]
 [-2.89817]
 [ 1.1219 ]]